# n-gram言語モデル
2-gramのモデルで前回のようにtrain、testを行う。

エントロピーを計算する。

2-gramだが、n-gramを想定してプログラムを作る。(本当か？)

In [29]:
# train-bigram.py
from collections import defaultdict

DEBUG = False
# データパス
train_data_path = "../../test/02-train-input.txt" if DEBUG else "../../data/wiki-en-train.word"
save_model_path = "train-input-model.txt" if DEBUG else "wiki-en-train-model.txt"

counts = defaultdict(lambda: 0)
context_counts = defaultdict(lambda: 0)

# テキストの読み込み
with open(train_data_path) as f:
    for line in f:
        words = line.strip().split()
        # 終端記号を追加
        words.append("</s>")
        # 文頭記号を追加
        words.insert(0, "<s>")
        for i in range(len(words) - 1):
            counts["{} {}".format(words[i], words[i+1])] += 1
            context_counts["{}".format(words[i])] += 1
            counts["{}".format(words[i+1])] += 1
            context_counts[""] += 1

ans_list = []
with open(save_model_path, mode="w") as f:
    for ngram, count in counts.items():
        context = ngram.split()
        if len(context) > 1:
            context = context[0]
        else:
            context = ""
        probability = counts[ngram] / context_counts[context]
        ans_list.append([ngram, probability])

    prob_list.sort(key=lambda x: x[0])
    for ngram, probability in prob_list:
        output = "{}\t{}\n".format(ngram, probability)
        f.write(output)


In [5]:
# test-bigram.py
import math
from collections import defaultdict

DEBUG = False
# データパス
model_path = "train-input-model.txt" if DEBUG else "wiki-en-train-model.txt"
test_data_path =  "../../test/02-test-input.txt" if DEBUG else "../../data/wiki-en-test.word"

lambda_1 = 0.95
lambda_2 = 0.80
V = 1000000
w = 0
h = 0

probs = defaultdict(lambda: 0)
with open(model_path) as f:
    for line in f:
        ngram, probability = line.strip().split("\t")
        probs[ngram] = float(probability)
probs

for lambda_1 in range(0, 100, 5):
    lambda_1 /= 100
    for lambda_2 in range(0, 100, 5):
        lambda_2 /= 100
        with open(test_data_path) as f:
            for line in f:
                words = line.strip().split()
                # 終端記号を追加
                words.append("</s>")
                # 文頭記号を追加
                words.insert(0, "<s>")
                w += len(words)
                for i in range(len(words) - 1):
                    p1 = lambda_1 * probs[words[i+1]] + (1 - lambda_1) / V
                    p2 = lambda_2 * probs["{} {}".format(words[i], words[i+1])] + (1 - lambda_2) * p1
                    h += -math.log(p2, 2)
            print("lambda_1: {}, lambda_2: {}, entropy: {}".format(lambda_1, lambda_2, h/w))

lambda_1: 0.0, lambda_2: 0.0, entropy: 19.2367065458073
lambda_1: 0.0, lambda_2: 0.05, entropy: 16.76224016625983
lambda_1: 0.0, lambda_2: 0.1, entropy: 15.804696114270119
lambda_1: 0.0, lambda_2: 0.15, entropy: 15.272480391271152
lambda_1: 0.0, lambda_2: 0.2, entropy: 14.925845567393617
lambda_1: 0.0, lambda_2: 0.25, entropy: 14.679313571346247
lambda_1: 0.0, lambda_2: 0.3, entropy: 14.494162899848126
lambda_1: 0.0, lambda_2: 0.35, entropy: 14.350093117096854
lambda_1: 0.0, lambda_2: 0.4, entropy: 14.235363289076048
lambda_1: 0.0, lambda_2: 0.45, entropy: 14.14269835369992
lambda_1: 0.0, lambda_2: 0.5, entropy: 14.067368616423922
lambda_1: 0.0, lambda_2: 0.55, entropy: 14.006206169569039
lambda_1: 0.0, lambda_2: 0.6, entropy: 13.9570706491031
lambda_1: 0.0, lambda_2: 0.65, entropy: 13.918552952661587
lambda_1: 0.0, lambda_2: 0.7, entropy: 13.889822053214163
lambda_1: 0.0, lambda_2: 0.75, entropy: 13.870579786234394
lambda_1: 0.0, lambda_2: 0.8, entropy: 13.861137333820084
lambda_1: 0.

In [4]:
!cat ../../test/02-train-answer.txt

</s>	0.250000
<s> a	1.000000
a	0.250000
a b	1.000000
b	0.250000
b c	0.500000
b d	0.500000
c	0.125000
c </s>	1.000000
d	0.125000
d </s>	1.000000
